In [1]:
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
import tridesclous as tdc

from tridesclous import DataIO, CatalogueConstructor, Peeler

c:\users\agustin\appdata\local\programs\python\python36\lib\site-packages\tridesclous\__init__.py:19: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "c:\users\agustin\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\agustin\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\agustin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\agustin\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
  

In [2]:
import os
import inspect
import sys

file_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
wdir = os.path.dirname(file_dir)
os.chdir(wdir)
sys.path.append(wdir)
sys.path.append(file_dir)
print(os.getcwd())

C:\Users\Agustin\Dropbox\Doctorado\NeuroData


In [3]:
os.chdir('18-09-28')

In [4]:
import shutil, glob
dirname = 'test_dir'
if os.path.exists(dirname):
    #remove is already exists
    shutil.rmtree(dirname)    
dataio = DataIO(dirname=dirname)

# feed DataIO
filenames = glob.glob('for_*.abf')

print(filenames)

['for_testing.abf']


In [5]:
dataio.set_data_source(type='Axon', filenames=filenames)
print(dataio)
catalogueconstructor = CatalogueConstructor(dataio=dataio)
print(catalogueconstructor)

DataIO <id: 2325472733448> 
  workdir: test_dir
  sample_rate: 5050.50505050505
  total_channel: 2
  channel_groups: 0 [IN5 IN6]
  nb_segment: 1
  length: 4439848
  durations: 879.1 s.
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  Signal pre-processing not done yet


In [6]:
catalogueconstructor.set_preprocessor_params(chunksize=2048,
            common_ref_removal=False,
            highpass_freq=100.,
            lowpass_freq=2000.,                                             
            lostfront_chunksize=64,
            peak_sign='-',
            relative_threshold=4,
            peak_span=0.03,
            )

In [7]:
catalogueconstructor.estimate_signals_noise(seg_num=0, duration=500.)
print(catalogueconstructor.signals_medians)
print(catalogueconstructor.signals_mads)

[0.20105487 0.14853983]
[ 9.037965 10.724172]


In [9]:
t1 = time.perf_counter()
catalogueconstructor.run_signalprocessor(duration=5000.)
t2 = time.perf_counter()

print('run_signalprocessor', t2-t1, 's')
print(catalogueconstructor)

run_signalprocessor 3.0871594762477486 s
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  nb_peak_by_segment: 19213
  cluster_labels [-11]



In [10]:
catalogueconstructor.extract_some_waveforms(n_left=-80, n_right=80, mode='rand', nb_max=20000, align_waveform=True)
print(catalogueconstructor)

compute_all_centroid 0.4010339502354512
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  nb_peak_by_segment: 19213
  some_waveforms.shape: (19213, 160, 2)
  cluster_labels [0]



In [11]:
catalogueconstructor.clean_waveforms(alien_value_threshold=1.)
print(catalogueconstructor)

compute_all_centroid 0.00010832549305916928
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  nb_peak_by_segment: 19213
  some_waveforms.shape: (19213, 160, 2)
  cluster_labels [-9]



In [56]:
n_left, n_right = catalogueconstructor.find_good_limits(mad_threshold = 1,)
print(catalogueconstructor)

compute_all_centroid 0.07197710599803031
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  nb_peak_by_segment: 9430
  some_waveforms.shape: (9430, 89, 2)
  cluster_labels [0]



In [13]:
t1 = time.perf_counter()
catalogueconstructor.extract_some_features(method='global_pca', n_components=10)
t2 = time.perf_counter()
print('project', t2-t1)
print(catalogueconstructor)

project 1.7564547208491206
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  nb_peak_by_segment: 19213
  some_waveforms.shape: (19213, 160, 2)
  some_features.shape: (19213, 10)
  cluster_labels [-9]



In [69]:
t1 = time.perf_counter()
catalogueconstructor.find_clusters(method='kmeans', n_clusters=8)
t2 = time.perf_counter()
print('find_clusters', t2-t1)
print(catalogueconstructor)

compute_all_centroid 0.05561213028431666
order_clusters waveforms_rms
find_clusters 0.36494692627866243
CatalogueConstructor
  chan_grp 0 - IN5 IN6
  nb_peak_by_segment: 9430
  some_waveforms.shape: (9430, 89, 2)
  some_features.shape: (9430, 6)
  cluster_labels [0 1 2 3 4 5 6 7]



In [14]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.CatalogueWindow(catalogueconstructor)
win.show()
app.exec_()

-1

In [71]:
catalogueconstructor.order_clusters(by='waveforms_rms')

#put label 0 to trash
mask = catalogueconstructor.all_peaks['cluster_label'] == 0
catalogueconstructor.all_peaks['cluster_label'][mask] = -1
catalogueconstructor.on_new_cluster()


#save the catalogue
catalogueconstructor.make_catalogue_for_peeler()

order_clusters waveforms_rms
make_catalogue 0.09549979874691417


In [72]:
initial_catalogue = dataio.load_catalogue(chan_grp=0)

peeler = Peeler(dataio)
peeler.change_params(catalogue=initial_catalogue)

t1 = time.perf_counter()
peeler.run()
t2 = time.perf_counter()
print('peeler.run', t2-t1)

print()
for seg_num in range(dataio.nb_segment):
    spikes = dataio.get_spikes(seg_num)
    print('seg_num', seg_num, 'nb_spikes', spikes.size)

100%|█████████████████████████████████████████████████████████████████████████████| 4335/4335 [00:06<00:00, 635.36it/s]


peeler.run 6.9270621231648875

seg_num 0 nb_spikes 21715


In [73]:

%gui  qt5
import  pyqtgraph  as pg
app = pg.mkQApp()
win = tdc.PeelerWindow(dataio=dataio, catalogue=initial_catalogue)
win.show()
app.exec_()

-1